In [ ]:
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


Generated with a GenAI

In [ ]:
import torch
import torch.nn as nn
import networkx as nx
from torch_geometric.utils import get_laplacian, to_dense_adj
from torch_geometric.utils import from_networkx
import numpy as np

# Step 1: Create a graph using NetworkX
G = nx.karate_club_graph()

# Step 2: Convert NetworkX graph to PyTorch Geometric data
graph = from_networkx(G)

# Step 3: Extract edge index and set edge weights
edge_index = graph.edge_index
edge_weight = torch.ones(edge_index.shape[1])  # All edges have weight 1

# Step 4: Compute Laplacian matrix
laplacian_edge_index, laplacian_edge_weight = get_laplacian(edge_index, edge_weight)

# Step 5: Convert to dense Laplacian matrix
L_dense = to_dense_adj(laplacian_edge_index, edge_attr=laplacian_edge_weight).squeeze()

# Step 6: Compute eigenvalues and eigenvectors
eigenvalues, U = torch.linalg.eigh(L_dense)


The rest is from myself

In [ ]:
class GraphFourierLoss(nn.Module):
  def __init__(self, U, sep_rate, lambda_=0.4, eps=10e-4):
    super(GraphFourierLoss, self).__init__()
    self.mse = nn.MSELoss()
    self.U = U
    self.sep_rate = sep_rate
    self.lambda_ = lambda_
    self.eps = eps

  @staticmethod
  def adjust(y, en, en_high, alpha):
    idx = torch.isin(en, en_high).nonzero()
    y[idx] = y[idx] * alpha
    return y

  def forward(self, predictions, target):
    y_hat = torch.matmul(self.U.T, predictions)
    y_hat_train = torch.matmul(self.U.T, target)

    E = torch.sum(torch.square(y_hat), dim=-1)
    E_train = torch.sum(torch.square(y_hat_train), dim=-1)

    E_sorted, _ = torch.sort(E)
    E_train_sorted, _ = torch.sort(E_train)

    segmenting_index = int(np.floor(E.size(0) * self.sep_rate))

    E_low = E_sorted[:segmenting_index]
    E_high = E_sorted[segmenting_index:]

    E_low_train = E_train_sorted[:segmenting_index]
    E_high_train = E_train_sorted[segmenting_index:]

    alpha = torch.sqrt(torch.mean(E_low) / (torch.mean(E_high) + self.eps)) * self.lambda_

    y_hat_prime = self.adjust(y_hat, E, E_high, alpha)
    y_hat_train_prime = self.adjust(y_hat_train, E_train, E_high_train, alpha)

    return self.mse(y_hat_prime, y_hat_train_prime)


In [ ]:
N = 34
d_k = 6

predictions = torch.rand(N, d_k)
target = torch.rand(N, d_k)

gfl = GraphFourierLoss(U, sep_rate=0.2)

print(gfl(predictions, target))


tensor(0.0100)
